<a href="https://colab.research.google.com/github/ko74dev/AI/blob/main/Dedect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачаем и распакуем набор данных для обучения:

In [1]:
!wget https://lodmedia.hb.bizmrg.com/case_files/789899/train_dataset_train.zip

--2022-08-14 12:25:29--  https://lodmedia.hb.bizmrg.com/case_files/789899/train_dataset_train.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 937581714 (894M) [application/zip]
Saving to: ‘train_dataset_train.zip’

train_dataset_train 100%[===================>] 894.15M   666KB/s    in 25m 42s 

2022-08-14 12:51:12 (594 KB/s) - ‘train_dataset_train.zip’ saved [937581714/937581714]



In [4]:
! unzip train_dataset_train.zip -x train

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/labels/24-11-2021_02-05-58_PM_human..txt  
  inflating: train/labels/24-11-2021_02-06-07_PM_car..txt  
  inflating: train/labels/24-11-2021_02-06-07_PM_head..txt  
  inflating: train/labels/24-11-2021_02-06-07_PM_human..txt  
  inflating: train/labels/24-11-2021_02-06-12_PM_car..txt  
  inflating: train/labels/24-11-2021_02-06-32_PM_car..txt  
  inflating: train/labels/24-11-2021_02-06-32_PM_face..txt  
  inflating: train/labels/24-11-2021_02-06-32_PM_head..txt  
  inflating: train/labels/24-11-2021_02-06-32_PM_human..txt  
  inflating: train/labels/24-11-2021_02-06-53_PM_car..txt  
  inflating: train/labels/24-11-2021_02-06-53_PM_head..txt  
  inflating: train/labels/24-11-2021_02-06-53_PM_human..txt  
  inflating: train/labels/24-11-2021_02-07-04_PM_car..txt  
  inflating: train/labels/24-11-2021_02-07-04_PM_head..txt  
  inflating: train/labels/24-11-2021_02-07-04_PM_human..txt  
  inflating: trai

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp train_dataset_train.zip /content/drive/MyDrive
# !rm train_dataset_train.zip

Подготовим данные для обучения и валидации

In [9]:
!mkdir output
!mkdir output/train
!mkdir output/val

In [10]:
from sklearn.model_selection import train_test_split
import os

train_names, val_names = train_test_split(
                            [f.split('.')[0] for f in os.listdir('train/images')], 
                                    test_size=0.15, random_state=22)

In [11]:
import shutil
classes = ['car', 'head', 'face', 'human', 'carplate']
def wt1(path):
    shutil.copy('train/images/' + name + '.jpg', f'output/{path}/')
    with open(f'output/{path}/'+name+'.txt', 'a') as fo:
        for file in os.listdir('train/labels'):
            if name in file:
                with open('train/labels/' + file) as fi:
                    for line in fi.readlines():
                        cl = classes.index(file.split('_')[-1].split('.')[0])
                        line = ' '.join([str(cl)]+line.split()[1:])+'\n'
                        fo.write(line)
for name in [f.split('.')[0] for f in os.listdir('train/images')]:
    if name in val_names: wt1('val')
    else: wt1('train')

Сформируем json файлы в формате COCO

In [12]:
from pathlib import Path
import imagesize
import json

In [13]:
# для раздела аннотаций
def create_image_annotation(file_path: Path, width: int, height: int, image_id: int):
    file_path = file_path.name
    image_annotation = {
        "file_name": file_path,
        "height": height,
        "width": width,
        "id": image_id}
    return image_annotation

def create_annotation_from_yolo_format(
    min_x, min_y, width, height, image_id, category_id, annotation_id):
    bbox = (float(min_x), float(min_y), float(width), float(height))
    area = width * height
    max_x = min_x + width
    max_y = min_y + height
    annotation = {
        "id": annotation_id,
        "image_id": image_id,
        "bbox": bbox,
        "area": area,
        "iscrowd": 0,
        "category_id": category_id,
        "segmentation": []}
    return annotation

In [14]:
# для раздела аннотаций и данных изображений
def get_images_info_and_annotations(path):
    path = Path(path)
    annotations = []
    images_annotations = []
    file_paths = sorted(path.rglob("*.jpg"))
    image_id = 0
    annotation_id = 1  # В формате COCO нумеррация классов начинается с '1'
    for file_path in file_paths:
        w, h = imagesize.get(str(file_path))
        image_annotation = create_image_annotation(
            file_path=file_path, width=w, height=h, image_id=image_id)
        images_annotations.append(image_annotation)

        label_file_name = f"{file_path.stem}.txt"
        annotations_path = file_path.parent / label_file_name

        if not annotations_path.exists(): continue

        with open(str(annotations_path), "r") as label_file:
            label_read_line = label_file.readlines()

        for line1 in label_read_line:
            label_line = line1
            category_id = (
                int(label_line.split()[0]) + 1)
            x_center = float(label_line.split()[1])
            y_center = float(label_line.split()[2])
            width = float(label_line.split()[3])
            height = float(label_line.split()[4])

            float_x_center = w * x_center
            float_y_center = h * y_center
            float_width = w * width
            float_height = h * height

            min_x = int(float_x_center - float_width / 2)
            min_y = int(float_y_center - float_height / 2)
            width = int(float_width)
            height = int(float_height)

            annotation = create_annotation_from_yolo_format(
                min_x,
                min_y,
                width,
                height,
                image_id,
                category_id,
                annotation_id)
            annotations.append(annotation)
            annotation_id += 1
        image_id += 1
    return images_annotations, annotations

Для обучения:

In [15]:
coco_format = {"images": [{}], "categories": [], "annotations": [{}]}
output_path = 'output/train/'
(coco_format["images"], coco_format["annotations"],
        ) = get_images_info_and_annotations(output_path)

for index, label in enumerate(classes):
    categories = {
        "supercategory": "Defect",
        "id": index + 1,  # ID starts with '1' .
        "name": label}
    coco_format["categories"].append(categories)
    with open(output_path+'/train.json', "w") as outfile:
        json.dump(coco_format, outfile, indent=4)
    print("OK!")

OK!
OK!
OK!
OK!
OK!


Для валидации

In [16]:
coco_format = {"images": [{}], "categories": [], "annotations": [{}]}
output_path = 'output/val/'
(coco_format["images"], coco_format["annotations"],
        ) = get_images_info_and_annotations(output_path)

for index, label in enumerate(classes):
    categories = {
        "supercategory": "Defect",
        "id": index + 1,
        "name": label}
    coco_format["categories"].append(categories)

    with open(output_path+'/val.json', "w") as outfile:
        json.dump(coco_format, outfile, indent=4)
    print("OK!")

OK!
OK!
OK!
OK!
OK!


Класс для датасета CocoDetection:

In [17]:
from torchvision import datasets

class CocoDetection(datasets.CocoDetection):
    def __init__(self, img_folder, feature_extractor, train=True):
        ann_file = os.path.join(img_folder, "train.json" if train else "val.json")
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img, target = super(CocoDetection, self).__getitem__(idx)

        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.feature_extractor(images=img, 
                                          annotations=target, 
                                          return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        target = encoding["labels"][0] # remove batch dimension

        return pixel_values, target

In [20]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.3 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Для выбранной модели для детекции (Yolos), YolosFeatureExtractor можно использовать для подготовки изображений.

In [21]:
from transformers import YolosFeatureExtractor

feature_extractor = YolosFeatureExtractor.from_pretrained(
    "hustvl/yolos-small", do_resize=True, size=512)

Датасеты для тренировачной и валидационной выборки для детекции в формате Coco

In [23]:
train_dataset = CocoDetection(img_folder='output/train', 
                              feature_extractor=feature_extractor)
val_dataset = CocoDetection(img_folder='output/val', 
                            feature_extractor=feature_extractor, train=False)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Размеры тренировачной и валидационной выборки:

In [24]:
len(train_dataset), len(val_dataset)

(1701, 301)

Создание DataLoader-ов, которые будут генировать данные для модели

In [46]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    pixel_values = [item[0] for item in batch]
    encoding = feature_extractor.pad(pixel_values, return_tensors="pt")
    labels = [item[1] for item in batch]
    batch = {}
    batch['pixel_values'] = encoding['pixel_values']
    batch['labels'] = labels
    return batch

train_dataloader = DataLoader(
            train_dataset, collate_fn=collate_fn, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=1)
batch = next(iter(train_dataloader))

Словарь соответствий: номер - имя класса

In [27]:
id2label = {k: v['name'] for k,v in train_dataset.coco.cats.items()}
id2label

{1: 'car', 2: 'head', 3: 'face', 4: 'human', 5: 'carplate'}

In [29]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 701 kB 8.6 MB/s 
     |████████████████████████████████| 419 kB 63.9 MB/s 
     |████████████████████████████████| 141 kB 72.9 MB/s 
     |████████████████████████████████| 5.9 MB 43.8 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


Для обучения модели YolosForObjectDetection воспользуемся pytorch_lightning

In [47]:
import pytorch_lightning as pl
from transformers import DetrConfig, YolosForObjectDetection
from torch.optim import AdamW

class Detr(pl.LightningModule):
    def __init__(self, lr=2.5e-5, weight_decay=1e-4):
        super().__init__()
        self.model = YolosForObjectDetection.from_pretrained("hustvl/yolos-small", 
                                                         num_labels=len(id2label),
                                                         ignore_mismatched_sizes=True)
        self.lr = lr
        self.weight_decay = weight_decay

    def forward(self, pixel_values):
        outputs = self.model(pixel_values=pixel_values)
        return outputs
     
    def common_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

        outputs = self.model(pixel_values=pixel_values, labels=labels)

        loss = outputs.loss
        loss_dict = outputs.loss_dict

        return loss, loss_dict

    def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        self.log("training_loss", loss)
        for k,v in loss_dict.items():
            self.log("train_" + k, v.item())
        return loss

    def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss)
        for k,v in loss_dict.items():
            self.log("validation_" + k, v.item())
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.lr,
                                  weight_decay=self.weight_decay)
        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

Обучение модели

In [48]:
model = Detr(lr=2.5e-5, weight_decay=1e-4)

from pytorch_lightning import Trainer

trainer = Trainer(gpus=1, 
                  max_steps=2000, 
                    gradient_clip_val=0.1, accumulate_grad_batches=4)
trainer.fit(model)

Some weights of YolosForObjectDetection were not initialized from the model checkpoint at hustvl/yolos-small and are newly initialized because the shapes did not match:
- class_labels_classifier.layers.2.weight: found shape torch.Size([92, 384]) in the checkpoint and torch.Size([6, 384]) in the model instantiated
- class_labels_classifier.layers.2.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:p

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2500` reached.


In [44]:
from torch import no_grad, cuda
with no_grad():
  for m in model.children():
    m.cuda()
    m.eval()
    # x = m(x)
    m.cpu()
    cuda.empty_cache()

In [45]:
import gc
cuda.empty_cache()
gc.collect()

176

Тестирование обученой модели

In [53]:
from pandas import DataFrame

def b2b(b):
    center_x, center_y, width, height = b
    return[center_x - width / 2,
           center_x + width / 2,
           center_y - height / 2,
           center_y + height / 2]

def get_test_labels_df(path):
    simple_solution = []
    for detection_file in [x for x in os.listdir(path) if 'txt' in x]:
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, center_x, center_y, width, height = val
            xmin, xmax, ymin, ymax = b2b([center_x, center_y, width, height])
            simple_solution.append([img_name, cls, xmin, xmax, ymin, ymax])
    return simple_solution

test_labels = get_test_labels_df('output/val/')
test_labels = DataFrame(test_labels, 
                   columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

In [54]:
test_labels['LabelName'] = test_labels.LabelName.astype(int)

In [55]:
from torch import device, cuda
device = device("cuda" if cuda.is_available() else "cpu")
model.to(device)
model.eval();

In [57]:
from PIL import Image
from tqdm import tqdm

solution = []
for name in tqdm([x for x in os.listdir('output/val') if 'jpg'in x][:300]):
    im = Image.open('output/val/'+name)
    pixel_values = feature_extractor(im, return_tensors="pt").pixel_values
    outputs = model(pixel_values.to(device))
    maxprobs = outputs.logits.softmax(-1)[0, :, :-1].max(-1)
    for i in range(len(maxprobs.indices)):
        if maxprobs.values[i] > .5:
            solution.append([name] + [(maxprobs.indices[i].item()-1)%5]+
                 [maxprobs.values[i].item()] + 
                 [x.item() for x in b2b(outputs.pred_boxes[0, [i]].cpu()[0])])

100%|██████████| 300/300 [01:00<00:00,  4.99it/s]


In [58]:
simple_solution = DataFrame(solution, 
               columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
simple_solution[['XMin','XMax','YMin','YMax']
             ] = simple_solution[['XMin','XMax','YMin','YMax']].clip(0,1)
simple_solution

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,23-11-2021_05-38-07_PM.jpg,0,0.965053,0.105039,0.191237,0.158907,0.231030
1,23-11-2021_05-38-07_PM.jpg,3,0.970547,0.606138,0.628866,0.265035,0.390576
2,23-11-2021_05-38-07_PM.jpg,0,0.965056,0.320690,0.449562,0.232706,0.306808
3,23-11-2021_05-38-07_PM.jpg,0,0.815412,0.106339,0.193815,0.159658,0.228810
4,23-11-2021_05-38-07_PM.jpg,0,0.980741,0.265057,0.502824,0.347597,0.522861
...,...,...,...,...,...,...,...
7317,24-11-2021_05-11-47_AM.jpg,0,0.813723,0.202973,0.257187,0.162605,0.209344
7318,24-11-2021_05-11-47_AM.jpg,0,0.874657,0.448519,0.514668,0.124720,0.187324
7319,24-11-2021_05-11-47_AM.jpg,0,0.928922,0.533009,0.602260,0.141010,0.208264
7320,24-11-2021_05-11-47_AM.jpg,0,0.948398,0.266704,0.311609,0.120722,0.164748


In [60]:
test_labels.to_csv('val.csv', index=False, sep=';')

In [62]:
!pip install map_boxes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
from map_boxes import mean_average_precision_for_boxes
mean_ap, average_precisions = mean_average_precision_for_boxes(test_labels, 
                                                               simple_solution, 
                                                               iou_threshold=0.5, 
                                                               verbose=False)
mean_ap, average_precisions

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.7/dist-packages/map_boxes/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


(0.20968040207377023,
 {'0': (0.7110145197895248, 2516.0),
  '1': (0.008162134199444218, 640.0),
  '2': (0.0, 117.0),
  '3': (0.3292253563798821, 644.0),
  '4': (0.0, 51.0)})

Формирование данных для тестирования

In [64]:
!wget https://lodmedia.hb.bizmrg.com/case_files/789899/test_dataset_test.zip
!unzip test_dataset_test.zip -x test

--2022-08-14 15:10:02--  https://lodmedia.hb.bizmrg.com/case_files/789899/test_dataset_test.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 455731722 (435M) [application/zip]
Saving to: ‘test_dataset_test.zip’

test_dataset_test.z 100%[===================>] 434.62M  16.5MB/s    in 89s     

2022-08-14 15:11:32 (4.88 MB/s) - ‘test_dataset_test.zip’ saved [455731722/455731722]

Archive:  test_dataset_test.zip
   creating: test/
   creating: test/images/
  inflating: test/images/0_23-11-2021_02-18-50_PM.jpg  
  inflating: test/images/0_23-11-2021_02-18-56_PM.jpg  
  inflating: test/images/0_23-11-2021_02-19-19_PM.jpg  
  inflating: test/images/0_23-11-2021_02-20-47_PM.jpg  
  inflating: test/images/0_23-11-2021_02-37-47_PM.jpg  
  inflating: test/images/0_23-11-2021_02-38-40_PM.jpg  
  inflating: test/images/

In [75]:
solution = []
for name in tqdm([x for x in os.listdir('test/images/') if 'jpg'in x]):
    im = Image.open('test/images/'+name)
    pixel_values = feature_extractor(im, return_tensors="pt").pixel_values
    outputs = model(pixel_values.to(device))
    maxprobs = outputs.logits.softmax(-1)[0, :, :-1].max(-1)
    for i in range(len(maxprobs.indices)):
        if maxprobs.values[i] > .5:
            solution.append([name] + [(maxprobs.indices[i].item()-1)%5]+
                 [maxprobs.values[i].item()] + 
                 [x.item() for x in b2b(outputs.pred_boxes[0, [i]].cpu()[0])])
test_solution = DataFrame(solution, 
               columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

  0%|          | 0/855 [00:00<?, ?it/s]

0_24-11-2021_05-49-06_AM.jpg


  0%|          | 1/855 [00:00<03:25,  4.16it/s]

24-11-2021_04-55-09_AM.jpg


  0%|          | 2/855 [00:00<03:26,  4.14it/s]

...

100%|█████████▉| 853/855 [03:00<00:00,  4.80it/s]

26-11-2021_04-47-01_AM.jpg
25-11-2021_02-12-53_PM.jpg


100%|█████████▉| 854/855 [03:00<00:00,  4.55it/s]

24-11-2021_06-24-01_AM.jpg


100%|██████████| 855/855 [03:01<00:00,  4.72it/s]


In [70]:
# test_solution = DataFrame(solution, 
#                columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [76]:
test_solution[['XMin','XMax','YMin','YMax']
             ] = test_solution[['XMin','XMax','YMin','YMax']].clip(0,1)

In [77]:
test_solution

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_04-55-09_AM.jpg,0,0.834969,0.310544,0.418685,0.043818,0.147340
1,24-11-2021_04-55-09_AM.jpg,0,0.880298,0.281482,0.380636,0.044835,0.147084
2,24-11-2021_04-55-09_AM.jpg,0,0.996688,0.391524,0.519056,0.069059,0.199037
3,24-11-2021_04-55-09_AM.jpg,0,0.977760,0.221707,0.283518,0.001689,0.062893
4,24-11-2021_04-55-09_AM.jpg,0,0.526885,0.389143,0.499639,0.067388,0.187654
...,...,...,...,...,...,...,...
23308,24-11-2021_06-24-01_AM.jpg,0,0.999666,0.053780,0.233607,0.486711,0.761449
23309,24-11-2021_06-24-01_AM.jpg,0,0.999125,0.149570,0.232119,0.222043,0.321650
23310,24-11-2021_06-24-01_AM.jpg,0,0.928047,0.417646,0.488560,0.071738,0.128898
23311,24-11-2021_06-24-01_AM.jpg,0,0.589113,0.513481,0.564651,0.029126,0.069692


In [78]:
test_solution.to_csv('sub1.csv', index=False, sep=';')